# Advent of Code 2023 - Day 10: **Pipe Maze**

### Preparation

In [98]:
import numpy as np
import heapq as hq
from queue import PriorityQueue
from dataclasses import dataclass

In [99]:
with open('test.txt', 'r') as f:
    data = np.array([[int(c) for c in list(line)] for line in f.read().splitlines()]).T
width, height = data.shape

In [100]:
class Node:
    pos : (int, int)
    g : np.float32 = np.inf
    f : np.float32 = np.inf
    parent : 'Node' = None
    neighbourrs : list['Node'] = None
    dir_last : (int, int) = (0,0)
    dir_count : int = 0
    
    def __init__(self, pos : (int, int)) -> None:
        self.pos = pos
        
    def __eq__(self, other):
        return self.pos == other.pos
    
    def __lt__(self, other):
        return self.g < other.g
    
    def __add__(self, other):
        return self.pos[0] + other.pos[0], self.pos[1] + other.pos[1]
    
    def __sub__(self, other):
        return self.pos[0] - other.pos[0], self.pos[1] - other.pos[1]
    
    def inbounds(self):
        return (0 <= self.pos[0] < width) and (0 <= self.pos[1] < height)

    def h(self, other):
        return abs(self.pos[0] - other.pos[0]) + abs(self.pos[1] - other.pos[1])
    
    def __str__(self) -> str:
        return f'N{self.pos}, g={self.g}, par={self.parent.pos if self.parent else None}, dir={self.dir_last}, dir_count={self.dir_count}'
    
    # def get_board_repr(self):
        
    
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

In [101]:
# def print_grid(grid):
#     for row in grid:
#         nodes = 

In [102]:
grid = [[Node((x, y)) for y in range(height)] for x in range(width)]
for x in range(width):
    for y in range(height):
        grid[x][y].neighbourrs = [grid[nx][ny] for nx, ny in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)] if (0 <= nx < width) and (0 <= ny < height)]

In [106]:
openlist : list[Node] = []
closedlist : list[Node] = []
start = grid[0][0]
start.g = 0
goal = grid[width-1][height-1]

num_grid = np.zeros((width, height))

def expand_node(current, openlist, closedlist):
    for successor in current.neighbourrs:
        dir = current - successor
        if current.dir_last == dir:
            if current.dir_count < 2:
                successor.dir_count = current.dir_count + 1
            else:
                continue
        if current.dir_last != dir:
            successor.dir_count = 0
            successor.dir_last = dir
        
        if successor in closedlist:
            continue
        tent_g = current.g + data[successor.pos]
        if successor in (node[1] for node in openlist):
            if tent_g >= successor.g:
                continue
        successor.parent = current
        successor.g = tent_g
        f = tent_g + successor.h(goal)
        
        index = None
        for i, node in enumerate(openlist):
            if node[1] == successor:
                index = i
                break
        if index:
            openlist[index] = (f, successor)
            hq.heapify(openlist)
        else:
            hq.heappush(openlist, (f, successor))
        
        num_grid[successor.pos] = f
            
hq.heappush(openlist, (0, start))

while openlist:
    current = hq.heappop(openlist)[1]
    print(current)
    if current == goal:
        break
    closedlist.append(current)
    expand_node(current, openlist, closedlist)
    

print('path:')
path = []
current = goal
while current.parent:
    print(current)
    path.append(current)
    current = current.parent
    
print(num_grid)


N(0, 0), g=0, par=None, dir=(1, 0), dir_count=0
N(0, 1), g=3, par=(0, 0), dir=(0, -1), dir_count=0
N(1, 0), g=4, par=(0, 0), dir=(-1, 0), dir_count=0
N(1, 1), g=5, par=(0, 1), dir=(0, -1), dir_count=0
N(2, 0), g=5, par=(1, 0), dir=(1, 0), dir_count=1
N(2, 1), g=6, par=(1, 1), dir=(0, -1), dir_count=0
N(0, 2), g=6, par=(0, 1), dir=(0, 1), dir_count=1
N(1, 2), g=7, par=(1, 1), dir=(-1, 0), dir_count=0
N(3, 0), g=8, par=(2, 0), dir=(-1, 0), dir_count=0
N(0, 3), g=9, par=(0, 2), dir=(0, -1), dir_count=0
N(2, 2), g=11, par=(2, 1), dir=(1, 0), dir_count=1
N(3, 1), g=11, par=(2, 1), dir=(0, -1), dir_count=0
N(1, 3), g=11, par=(1, 2), dir=(-1, 0), dir_count=0
N(4, 0), g=12, par=(3, 0), dir=(1, 0), dir_count=1
N(0, 4), g=13, par=(0, 3), dir=(1, 0), dir_count=1
N(0, 5), g=14, par=(0, 4), dir=(0, -1), dir_count=0
N(4, 1), g=15, par=(3, 1), dir=(0, -1), dir_count=0
N(5, 0), g=15, par=(4, 0), dir=(-1, 0), dir_count=0
N(2, 3), g=15, par=(2, 2), dir=(0, -1), dir_count=1
N(3, 2), g=16, par=(2, 2), dir